# M365 Incident Response — Compromised Email (App-Only)

This notebook is a starter playbook for automated incident response (IR) focused on a potentially compromised Microsoft 365 mailbox. It is written for the .NET PowerShell kernel (PowerShell via .NET Interactive) and uses app-only (client credentials) authentication against Microsoft Graph so it can be run unattended by a service account or automation pipeline.

Security note: do NOT commit client secrets or certificates into source control. Store secrets in a secure location (Key Vault, files with strict permissions, or environment variables) and restrict access to the notebook and host environment.